In [1]:
import pandas as pd
import numpy as np
import math as mt

In [2]:
df = pd.read_csv("Alpha_bank.csv")
df.shape

(1000, 8)

In [3]:
#---------------DATA CLEANSING---------------#

#Ubah admin. jadi admin di kolom 'Job'
df.loc[df['Job'] == 'admin.', 'Job'] = 'admin'
df.head()

#Ubah int pada column 'Age' menjadi child (0-12), young adult (13-18), adult (19-59), senior (>60)
for index, row in df.iterrows():
    if (row['Age'] >= 0 and row['Age'] <= 12):
#         print('Child')
        df.loc[index,'Age'] = 'child'
    elif (row['Age'] >= 13 and row['Age'] <= 18):
#         print('Young Adult')
        df.loc[index,'Age'] = 'young adult'
    elif (row['Age'] >= 19 and row['Age'] <= 59):
#         print('Adult')
        df.loc[index,'Age'] = 'adult'
    elif (row['Age'] >= 60):
#         print('Senior')
        df.loc[index,'Age'] = 'senior'
    print(index)
    
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,adult,housemaid,married,Primary_Education,no,no,no,no
1,adult,services,married,Secondary_Education,no,yes,no,no
2,adult,admin,married,Primary_Education,no,no,no,no
3,adult,services,married,Secondary_Education,no,no,yes,no
4,adult,admin,married,Professional_Education,no,no,no,no
...,...,...,...,...,...,...,...,...
995,adult,blue-collar,married,Professional_Education,no,no,no,no
996,adult,technician,married,Secondary_Education,no,no,no,no
997,adult,blue-collar,married,Tertiary_Education,no,no,yes,no
998,adult,blue-collar,single,Secondary_Education,no,yes,no,no


In [4]:
#---------------Data Split---------------#

# 30% test data
test_amount = int(round(df.shape[0]*0.3,0))

# 70% training data
train_amount = int(round(df.shape[0]*0.7,0))
print('test : ',test_amount)
print('training : ',train_amount)

df_training = df.head(train_amount)
df_test = df.tail(test_amount)



test :  300
training :  700


In [9]:
#---------------Check Data Distribution---------------#

# print(df_training['Job'].value_counts())
# print()
# print(df_test['Job'].value_counts())

married     491
single      129
divorced     80
Name: Marital_Status, dtype: int64

In [6]:
#---------------Entropy---------------#

df_header = list(df.columns.values)
# print(df_header)

def calculate_total_attr(group, unique_value):    
    l_total = list()
    
    for x in unique_value:
        total = 0
        l_total.append(group['no'][x] + group['yes'][x])
    
    total_per_attribute = dict(zip(unique_value, l_total))
    
    return total_per_attribute
        
def group_check(group,unique_value):
    arr_subs = ['no','yes']
    
    for x in unique_value:
        for i in arr_subs:
            if np.isnan(group[i][x]):
                group[i][x] = 0
                
    return group

def universal_entropy(group, unique_value, total_attr):
    yes = 0
    no = 0
    
    temp_no = df_training[df_training['Subscribed']=='no']
    temp_yes = df_training[df_training['Subscribed']=='yes']

    no = temp_no.shape[0]
    yes = temp_yes.shape[0]
    
#     print(yes/total_attr)
#     print(math.log((yes/total_attr),2))
    
#     print((-1*(yes/total_attr)*math.log((yes/total_attr),2)) + (-1*(no/total_attr)*math.log((no/total_attr),2)))
        
    return (-1*(yes/total_attr)*mt.log((yes/total_attr),2)) + (-1*(no/total_attr)*mt.log((no/total_attr),2))

def entropy_calculation(column_name):
    grouping_result = df_training.groupby([column_name,df_header[7]]).size().unstack()
    total_attr = 21334
    
    if (column_name == 'Age'):
        unique_value = df_training.Age.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Job'):
        unique_value = df_training.Job.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Marital_Status'):
        unique_value = df_training.Marital_Status.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Education'):
        unique_value = df_training.Education.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Default_Credit'):
        unique_value = df_training.Default_Credit.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Housing_Loan'):
        unique_value = df_training.Housing_Loan.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Personal_Loan'):
        unique_value = df_training.Personal_Loan.unique()
        grouping_result = group_check(grouping_result,unique_value)
    elif (column_name == 'Subscribed'):
        unique_value = df_training.Personal_Loan.unique()
        grouping_result = group_check(grouping_result,unique_value)
        
#     print(unique_value)
    if column_name != 'Subscribed':
        total = calculate_total_attr(grouping_result, unique_value)
        
        entropy = dict()
        for x in total:
            
            if grouping_result['yes'][x] == 0 or grouping_result['no'][x] == 0:
                entropy[x] = 0
            elif grouping_result['yes'][x] != 0 and grouping_result['no'][x] != 0:
                first_no = grouping_result['no'][x]/total[x]
                second_no = mt.log((grouping_result['no'][x]/total[x]),2)
                third_yes = grouping_result['yes'][x]/total[x]
                fourth_yes = mt.log((grouping_result['yes'][x]/total[x]),2)
                attr_entropy_value = (-1*(first_no*second_no)) + (-1*(third_yes*fourth_yes))
                entropy[x] = attr_entropy_value

        entropy_value_spec.append([column_name,entropy])

        column_entropy_value = 0
        for x in total:
            column_entropy_value += (total[x]/total_attr)*entropy[x]
#             print(x)
#             print((total[x]/total_attr)*entropy[x])
#         print(column_entropy_value)

        universal_value = universal_entropy(grouping_result, unique_value, total_attr)
#         print('uni_value : ',universal_value)
        all_entropy_value[column_name] = universal_value - column_entropy_value
        
def find_entropy(df_header):
    for column_name in df_header:
#         print(column_name)
        entropy_calculation(column_name)
    
#     sorted_gini = {k: v for k, v in sorted(all_gini_value.items(), key=lambda item: item[1])}
    sorted_entropy = sorted(all_entropy_value, key=all_entropy_value.get, reverse=True)
    return sorted_entropy[:3]
    
gini_value_spec = list()
entropy_value_spec = list()
all_entropy_value = dict()
l_entropy = find_entropy(df_header)

# a1 = df_training[df_training['Age']=='adult']
# a2 = df_training[df_training['Age']=='senior']
# a3 = df_training[df_training['Age']=='young adult']

# j1 = df_training[df_training['Education']=='Primary_Education']
# j2 = df_training[df_training['Education']=='Secondary_Education']
# j3 = df_training[df_training['Education']=='Professional_Education']
# j4 = df_training[df_training['Education']=='Tertiary_Education']


# mM = df_training[df_training['Marital_Status']=='married']
# mS = df_training[df_training['Marital_Status']=='single']
# mD = df_training[df_training['Marital_Status']=='divorced']

# print(a1.shape[0], a2.shape[0], a3.shape[0])
# print(j1.shape[0], j2.shape[0], j3.shape[0], j4.shape[0])
# print(mM.shape[0], mS.shape[0], mD.shape[0])
# print()
# print(entropy_value_spec)

# print()
print(l_entropy)
# print(wow)
# print('meng')



['Job', 'Education', 'Marital_Status']


In [7]:
#---------------Getting Data Ready For Model---------------#

#training data
new_df_training = df_training[[l_entropy[0], l_entropy[1], l_entropy[2],'Subscribed']].copy()

#testing data tanpa class
new_df_test = df_test[[l_entropy[0], l_entropy[1], l_entropy[2],'Subscribed']].copy()
nan_value = "NaN"
new_df_test['Subscribed']= nan_value

#class testing data
answer = df_test[['Subscribed']].copy()

In [10]:
#---------------Feature Engineering---------------#


#Age adalah ordinal category ("child","young adult","adult","senior")
new_df_training = new_df_training.replace(to_replace =("single","married","divorced"), 
                 value =(1,2,3))

new_df_test = new_df_test.replace(to_replace =("single","married","divorced"), 
                 value =(1,2,3)) 


new_df_test

,Job,Education,Marital_Status,Subscribed
700,admin,Secondary_Education,1,NaN
701,admin,Secondary_Education,2,NaN
702,technician,Professional_Education,2,NaN
703,blue-collar,Secondary_Education,2,NaN
704,services,Primary_Education,3,NaN
...,...,...,...,...
995,blue-collar,Professional_Education,2,NaN
996,technician,Secondary_Education,2,NaN
997,blue-collar,Tertiary_Education,2,NaN
998,blue-collar,Secondary_Education,1,NaN


In [11]:
#Education adalah ordinal category ("Primary_Education","Secondary_Education","Tertiary_Education","Professional_Education")
new_df_training = new_df_training.replace(to_replace =("Primary_Education","Secondary_Education","Tertiary_Education","Professional_Education"), 
                 value =(1,2,3,4))

new_df_test = new_df_test.replace(to_replace =("Primary_Education","Secondary_Education","Tertiary_Education","Professional_Education"), 
                 value =(1,2,3,4))

new_df_test

,Job,Education,Marital_Status,Subscribed
700,admin,2,1,NaN
701,admin,2,2,NaN
702,technician,4,2,NaN
703,blue-collar,2,2,NaN
704,services,1,3,NaN
...,...,...,...,...
995,blue-collar,4,2,NaN
996,technician,2,2,NaN
997,blue-collar,3,2,NaN
998,blue-collar,2,1,NaN


In [12]:
#Job adalah non ordinal category, di split menggunakan pd.get_dummies
new_df_training = pd.get_dummies(new_df_training, columns=["Job"])

new_df_test = pd.get_dummies(new_df_test, columns=["Job"])

new_df_test

,Education,Marital_Status,Subscribed,Job_admin,Job_blue-collar,Job_entrepreneur,Job_housemaid,Job_management,Job_retired,Job_self-employed,Job_services,Job_student,Job_technician,Job_unemployed
700,2,1,NaN,1,0,0,0,0,0,0,0,0,0,0
701,2,2,NaN,1,0,0,0,0,0,0,0,0,0,0
702,4,2,NaN,0,0,0,0,0,0,0,0,0,1,0
703,2,2,NaN,0,1,0,0,0,0,0,0,0,0,0
704,1,3,NaN,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,2,NaN,0,1,0,0,0,0,0,0,0,0,0
996,2,2,NaN,0,0,0,0,0,0,0,0,0,1,0
997,3,2,NaN,0,1,0,0,0,0,0,0,0,0,0
998,2,1,NaN,0,1,0,0,0,0,0,0,0,0,0


In [13]:
new_df_training = new_df_training[['Job_admin', 'Job_blue-collar',
 'Job_entrepreneur', 'Job_housemaid', 'Job_management', 'Job_retired',
 'Job_self-employed', 'Job_services', 'Job_student', 'Job_technician',
 'Job_unemployed', 'Education', 'Marital_Status', 'Subscribed']].copy()

new_df_test = new_df_test[['Job_admin', 'Job_blue-collar',
 'Job_entrepreneur', 'Job_housemaid', 'Job_management', 'Job_retired',
 'Job_self-employed', 'Job_services', 'Job_student', 'Job_technician',
 'Job_unemployed', 'Education','Marital_Status', 'Subscribed']].copy()

print(new_df_training)
print()
print(new_df_test)

     Job_admin  Job_blue-collar  Job_entrepreneur  Job_housemaid  \
0            0                0                 0              1   
1            0                0                 0              0   
2            1                0                 0              0   
3            0                0                 0              0   
4            1                0                 0              0   
..         ...              ...               ...            ...   
695          1                0                 0              0   
696          0                0                 0              0   
697          0                0                 0              1   
698          0                1                 0              0   
699          1                0                 0              0   

     Job_management  Job_retired  Job_self-employed  Job_services  \
0                 0            0                  0             0   
1                 0            0             

In [14]:
#---------------Model---------------#
count_subs = new_df_training[new_df_training['Subscribed']=='yes']
count_nsubs = new_df_training[new_df_training['Subscribed']=='no']
print(count_subs.shape[0])
print(count_nsubs.shape[0])

i=700
j=0

data3 = []
for i in range(len(new_df_test)):
    data = []
    data2 = []
    print(i)
    for j in range(len(new_df_training)):
        data.append(mt.sqrt(mt.pow(new_df_test.iloc[i][0] - new_df_training.iloc[j][0], 2) + mt.pow(
            new_df_test.iloc[i][1] - new_df_training.iloc[j][1], 2) + mt.pow(
            new_df_test.iloc[i][2] - new_df_training.iloc[j][2], 2) + mt.pow(
            new_df_test.iloc[i][3] - new_df_training.iloc[j][3], 2) + mt.pow(
            new_df_test.iloc[i][4] - new_df_training.iloc[j][4], 2) + mt.pow(
            new_df_test.iloc[i][5] - new_df_training.iloc[j][5], 2) + mt.pow(
            new_df_test.iloc[i][6] - new_df_training.iloc[j][6], 2) + mt.pow(
            new_df_test.iloc[i][7] - new_df_training.iloc[j][7], 2) + mt.pow(
            new_df_test.iloc[i][8] - new_df_training.iloc[j][8], 2) + mt.pow(
            new_df_test.iloc[i][9] - new_df_training.iloc[j][9], 2) + mt.pow(
            new_df_test.iloc[i][10] - new_df_training.iloc[j][10], 2) + mt.pow(
            new_df_test.iloc[i][11] - new_df_training.iloc[j][11], 2) + mt.pow(
            new_df_test.iloc[i][12] - new_df_training.iloc[j][12], 2)))
    rate = pd.DataFrame(data, columns=['Rate']).sort_values(by='Rate')
    for k in range(13):
        data2.append(new_df_training['Subscribed'][rate.index[k]])
    data3.append(data2)

print()
# print(data3)

data4 = []

for l in range(len(new_df_test)):
    yes = 0
    no = 0
    for m in range(13):
        if data3[l][m] == 'yes':
            yes += 1
        elif data3[l][m] == 'no':
            no += 1
    if (yes >= no) :
        highest = "yes"
    elif (no >= yes):
        highest = "no"

    data4.append(highest)

for n in range(len(new_df_test)):
    new_df_test.at[n+700,'Subscribed'] = data4[n]

print(new_df_test)
#------

print(testing_2.shape[0])

12
688
0


C:\Users\RUSWANTO\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in ubyte_scalars


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyError: 'Age'

In [68]:
#------Cek akurasi-------#

idex = 700
sama=0
tidak=0

for idex in range(len(new_df_test)):
    if(new_df_test.iloc[idex]["Subscribed"] == answer.iloc[idex]["Subscribed"]):
        sama = sama + 1
    else:
        tidak = tidak + 1

        
akurasi_benar = sama / len(new_df_test) * 100
akurasi_salah = tidak / len(new_df_test) * 100

print("Akurasi Benar: ", akurasi_benar)
print()
print("Akurasi Salah: ", akurasi_salah)

Akurasi Benar:  97.33333333333334

Akurasi Salah:  2.666666666666667


In [53]:
#---------------Visualisasi---------------#

True